In [ ]:
import pandas as pd
from fpt.path import DTFR, DATA

In [ ]:
# DATA_CATEGORY = "df_aihub_family_file"
DATA_CATEGORY = "df_aihub_sample_file"

df_aihub_file = pd.read_csv(DTFR / f"{DATA_CATEGORY}.csv", index_col="key")
df_aihub_file.head()

In [ ]:
group_dict = {
    "a": 1,
    "b": 2,
    "c": 3,
    "d": 4,
    "e": 5,
    "f": 6,
    "g": 7,
    "h": 8,
}


def convert_age_group(x):
    age_group = x.name.split('_')[-1]
    if len(age_group) == 1:
        age_group += "1"
    group = age_group[-2:-1]
    index = age_group[-1:]
    if group in group_dict:
        group_index = group_dict[group]
    else:
        group_index = 9
        print(x)
    return f"{group_index}0{index}"

In [ ]:
import re
p = re.compile(r'[^0-9-]')

In [ ]:
def convert_camera_degree(x):
    tag = x.name.split("_")
    degree = int(tag[-2]) + 90
    degree //= 45
    env = tag[-1]
    if env.upper() == "CAM":
        env = "06"
    else:
        env = p.sub("", env)
    env = int(env) * 1000
    if len(f"{(env+abs(degree))}") > 4 or len(f"{(env+abs(degree))}") == 0:
        return None
    if f"{(env+abs(degree))}" == "0":
        return "1000"  # F0028_IND_M_42_-90_0.JPG -> ..._01.JPG
    return f"{(env+abs(degree))}"

In [ ]:
category = "Age"
is_category = df_aihub_file.category == category
selected_index = df_aihub_file.index[is_category]
target_func = convert_age_group
df_aihub_file.loc[selected_index, "option"] = df_aihub_file[is_category].apply(
    target_func, axis=1
)

In [ ]:
category = "Individuals"
is_category = df_aihub_file.category == category
selected_index = df_aihub_file.index[is_category]
target_func = convert_camera_degree
df_aihub_file.loc[selected_index, "option"] = df_aihub_file[is_category].apply(
    target_func, axis=1  # type:ignore
)

In [ ]:
category = "Family"
is_category = df_aihub_file.category == category
selected_index = df_aihub_file.index[is_category]
df_aihub_file.loc[selected_index, "option"] = df_aihub_file[is_category].apply(
    lambda x:str(x.name.split("_")[-1]), axis=1
)

In [ ]:
df_aihub_file.groupby("category").option.apply(list).to_frame()

In [ ]:
df_aihub_file[df_aihub_file.category == 'Individuals'].option.unique()

In [ ]:
target = DATA / "dataframe" / f"{DATA_CATEGORY}.csv"
pd.DataFrame(df_aihub_file).to_csv(target, index_label="key")